# Implementation and Evaluation of Serverless Applications in the AWS Cloud Environment

## Structure 1 - Serverless AWS Web Application with Amplify, Lambda, IAM, API Gateway and DynamoDB

This project is a simple web application built from scratch using five key AWS services: Amplify, Lambda, IAM, API Gateway, and DynamoDB.

- **Amplify**: Used for deploying the frontend of the web application. Amplify provides an easy-to-use interface for connecting the app to AWS services and continuous deployment. It allows quick deployment of the frontend, automatic scaling, and easy integrations with other AWS services.
  
- **Lambda**: Acts as the backend, allowing you to execute code without provisioning or managing servers. Lambda functions are triggered by API Gateway and perform the necessary logic to fulfill user requests. It enables serverless execution of backend logic with high scalability and no infrastructure management.
  
- **IAM (Identity and Access Management)**: Ensures secure access control between AWS services. IAM roles and policies manage permissions for services like API Gateway, Lambda, and DynamoDB, ensuring the right service has the appropriate access level. It provides fine-grained access control to ensure that only the appropriate services and users can interact with each part of the application.
  
- **API Gateway**: Exposes RESTful APIs that act as the entry point for the frontend to communicate with the Lambda functions. API Gateway securely handles and routes requests between the frontend and the backend. It acts as a secure and scalable API interface for managing HTTP requests from the frontend.
  
- **DynamoDB**: Serves as the application's database, providing fast and scalable NoSQL storage for the web application’s data. It offers fast, flexible, and reliable NoSQL data storage for handling the application’s data requirements.

The web application frontend is deployed via AWS Amplify, which handles hosting and automatically connects it to other AWS resources like API Gateway and Lambda. When a user interacts with the web application, API Gateway triggers a Lambda function to handle requests (e.g., fetching or updating data). Lambda processes the request and performs the necessary operations. The Lambda function accesses DynamoDB, where the application data is stored, using secure access provided by IAM. IAM policies ensure that the API Gateway can trigger the Lambda functions, and that the Lambda functions can read/write data in DynamoDB.

The prerequisite is to have an AWS account with proper permissions for using Amplify, Lambda, IAM, API Gateway, and DynamoDB.



First we create a file named `index.html` with the following content:

``` html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Cloud Resource Calculator</title>
</head>
<body>
    <h1>Cloud Resource Calculator</h1>
    <p>Enter the number of users, and we'll calculate the cloud resources needed!</p>
    
    <form id="cloudForm">
        <label for="users">Number of users:</label>
        <input type="number" id="users" name="users" required>
        <button type="submit">Calculate Resources</button>
    </form>
    
    <p id="result"></p>

    <script>
        document.getElementById('cloudForm').addEventListener('submit', function(event) {
            event.preventDefault();
            const users = document.getElementById('users').value;

            fetch('https://your-api-gateway-endpoint', {
                method: 'POST',
                body: JSON.stringify({ 'users': users }),
                headers: {
                    'Content-Type': 'application/json'
                }
            })
            .then(response => response.json())
            .then(data => {
                document.getElementById('result').textContent = 'Resources needed: ' + data.resources;
            })
            .catch(error => console.error('Error:', error));
        });
    </script>
</body>
</html>
```

Then we zip this file.

We go to **Amplify**. Select `Deploy an app` -> `Deploy without Git`.

App name: `CloudCalc`.

Drag and drop the zipped index file.

Now we have a domain: `https://staging.d31thqf2f58vpx.amplifyapp.com/`

<img src="figs/structure1_01.png"  style="width:85%;"/>

Go to **Lambda**. Select "Create function".

Function name: `cloudResourceEstimator`

Runtime: `Python 3.12`.

Select `Create function`.

Modify the 'Lambda function' to the following:

```python
import json
import boto3
from time import gmtime, strftime
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('CloudResourceTable')
now = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())

def lambda_handler(event, context):
    users = int(event['users'])
    
    vm_count = users // 10
    storage = Decimal(users * 0.5)
    
    response = table.put_item(
        Item={
            'ID': str(users),
            'VMCount': vm_count,
            'StorageGB': storage,
            'Timestamp': now
        }
    )

    return {
        'statusCode': 200,
        'body': json.dumps({'resources': f'{vm_count} VMs, {storage} GB Storage'})
    }

```

Save the code and click on `Deploy`.

Go to `API Gateway`.

Select "Create API" -> "Rest API"

API Name: `CloudCalcAPI`

Click on 'Create API'.

Create Method:

'POST'

'Lambda function': `arn:aws:lambda:us-east-1:093511299840:function:cloudResourceEstimator`.

Click on `Enable CORS`.

Click on `Deploy API`.

Click to '*New stage'

Stage name: `dev`

Invoke URL: `https://wdmhkfiyub.execute-api.us-east-1.amazonaws.com/dev`

Go to **DynamoDB** -> 'Create table'.

Table name: `CloudResourceTable`.

Partition key: `ID`

Amazon Resource Name (ARN) is `arn:aws:dynamodb:us-east-1:093511299840:table/CloudResourceTable`

We also need to change the IAM role to allow DynamoDB full access.

Go back to the Lambda Function.

Click on `Test` and `Configure test event`.

Event name: `testEvent`.

Event JSON:

```json
{
  "users": "100"
}
```

Click on `Test`.

The result is:

Response:
```
{
  "statusCode": 200,
  "body": "{\"resources\": \"10 VMs, 50 GB Storage\"}"
}
```

Now, if we explore table items on DynamoDB, we get:

<img src="figs/structure1_02.png"  style="width:65%;"/>

Modify the HTML code to:

```html
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Cloud Resource Calculator</title>
    <!-- Styling for the client UI -->
    <style>
    h1 {
        color: #FFFFFF;
        font-family: system-ui;
        margin-left: 20px;
    }
    body {
        background-color: #222629;
    }
    label {
        color: #86C232;
        font-family: system-ui;
        font-size: 20px;
        margin-left: 20px;
        margin-top: 20px;
    }
    button {
        background-color: #86C232;
        border-color: #86C232;
        color: #FFFFFF;
        font-family: system-ui;
        font-size: 20px;
        font-weight: bold;
        margin-left: 30px;
        margin-top: 20px;
        width: 140px;
    }
    input {
        color: #222629;
        font-family: system-ui;
        font-size: 20px;
        margin-left: 10px;
        margin-top: 20px;
        width: 100px;
    }
    </style>
    <script>
        // callAPI function that takes the number of users as input
        var callAPI = (users) => {
            // instantiate a headers object
            var myHeaders = new Headers();
            // add content type header to object
            myHeaders.append("Content-Type", "application/json");
            // using built-in JSON utility package to turn object to string and store in a variable
            var raw = JSON.stringify({ "users": users });
            // create a JSON object with parameters for API call and store in a variable
            var requestOptions = {
                method: 'POST',
                headers: myHeaders,
                body: raw,
                redirect: 'follow'
            };
            // make API call with parameters and use promises to get the response
            fetch("https://wdmhkfiyub.execute-api.us-east-1.amazonaws.com/dev", requestOptions)
            .then(response => response.text())
            .then(result => alert(JSON.parse(result).body)) // showing result using alert
            .catch(error => console.log('error', error));
        }
    </script>
</head>
<body>
    <h1>CLOUD RESOURCE CALCULATOR</h1>
    <form>
        <label>Number of Users:</label>
        <input type="number" id="users">
        <!-- set button onClick method to call function we defined passing input values as parameters -->
        <button type="button" onclick="callAPI(document.getElementById('users').value)">CALCULATE</button>
    </form>
</body>
</html>

```

Zip it and update it on `Amplify`.

Modify the zip file on Amplify.

New link: `https://staging.d3mloeaccjdhj4.amplifyapp.com/`

<img src="figs/structure1_03.png"  style="width:75%;"/>

## Structure 2 - Serverless Web Application with AWS S3, API Gateway, Lambda Function, and DynamoDB

A web application using HTTP API will be deployed. This API includes various routes, such as `options`, `get`, `put`, and `delete`, corresponding to different paths. 

The setup includes a Lambda function and a DynamoDB table. To make it accessible to users, a static website hosted on Amazon S3 will be deployed. This static website will serve as the front-end interface for users, who can interact with the API via their web browsers.

The static website will contain a form where users can enter and submit information. The form will communicate with the public endpoint of the HTTP API, which will pass the data to the Lambda function. The Lambda function will process the request, and the processed data will be stored in a DynamoDB table.

1. A user interacts with the static website hosted on S3.
2. The website communicates with the HTTP API.
3. The API sends the request to the Lambda function.
4. The Lambda function processes the request and stores the result in the DynamoDB table.

The form that's generated will also be populated with information from the table, so it will always be returned back and viewable.

<img src="figs/serverless_webapp.png"  style="width:85%;"/>

### Create a Stack

On Amazon Cloud Formation, we create a Stack from an uploaded template `code/aws-backend-creation-cftemplate.yaml`

Stack name: `WebApp`

From this template, we can visualize the Application Composer canvas:

<img src="figs/application_composer_canvas.png"  style="width:85%;"/>

We need to check the field "I acknowledge that AWS CloudFormation might create IAM resources".

Click `Submit`.

### Create an S3 Bucket

On S3, create a bucket.

Bucket name: `frontend-website-00`

After creating the bucket, click on it and select "Properties". On "Static website hosting", click on "Edit". Select `Enable`.

Index document: `Index.html`.

Save changes.

Click on "Permissions". On "Block public access (bucket settings)", click on "Edit". Disable `Block all public access`.

Save changes.

Then Edit "Bucket policy". Insert the following code policy:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::frontend-website-00/*"
        }
    ]
}

```

The Bucket ARN (`arn:aws:s3:::frontend-website-00`) is the field in "Resource" plus "/*".

The bucket policy, written in JSON, provides access to the objects stored in the bucket. Now we have a bucket policy, which allows GetObjects (that is, we can have public reads of objects within the bucket).

Go to "Cross-origin resource sharing (CORS)" and edit it. Insert this:

```json
[
    {
        "AllowedHeaders": [
            "*"
        ],
        "AllowedMethods": [
            "GET"
        ],
        "AllowedOrigins": [
            "*"
        ],
        "ExposeHeaders": []
    }
]
```

The CORS configuration, written in JSON, defines a way for client web applications that are loaded in one domain to interact with resources in a different domain. This CORS policy allows any headers, the `GET` method, and any origins.





### Configure API Gateway

Go to "API Gateway". You can see that Cloud Formation created an API called `items-api`. Click on it and go to "Stages". Click on "Create".

Name: `prod`.

Click on "create".

Go to "Integrations" > "Manage integrations" > "Create".

Attach this integration to a route: `$default`

Integration type: `Lambda function`

AWS Region: `us-east-1` (same for every region asked before)

Lambda function: select the option below

`arn:aws:lambda:us-east-1:093511299840:function:WebApp-ItemLambdaFunction-gphsBpHEDO06`

Click on "Create".

Copy the "Integration ID": `4gt0yqu`.

Click on "Routes" > "Create a route". We'll create different routes for different methods. In "Route and method", add the following:

`GET` -> `/items`

`PUT` -> `/items`

`OPTIONS` -> `/items`

`GET` -> `/items/{id}`

`DELETE` -> `/items/{id}`

`OPTIONS` -> `/items/{id}`

So we have:

```asciidoc
$default
├── /items
│   ├── PUT
│   ├── OPTIONS
│   ├── GET
│   └── /{id}
│       ├── OPTIONS
│       ├── GET
│       └── DELETE
```

Then we select each one of these, choose "Attach integration" > "Choose an existing integration". Then choose the one created before (remember Integration ID) and click on "Attach integration"

<img src="figs/routes_for_items_api.png"  style="width:15%;"/>

Go to "CORS" > "Configure".

Access-Control-Allow-Origin: `https://frontend-website-00.s3.amazonaws.com`

Access-Control-Allow-Headers: `*`

Access-Control-Allow-Methods: `*`

Access-Control-Expose-Headers: `*`

Access-Control-Max-Age: `96400`

Access-Control-Allow-Credentials: `yes`

Click on "Save". Click on "Deploy".

Stage: `prod`

In the "Stages" section in the left-hand panel, we can find `prod`'s invoke URL: `https://txi4irodvl.execute-api.us-east-1.amazonaws.com/prod`

### Client-side code

On our `code/client/src/config.ts` file, we need to have the invoke URL:

```typescript
const apiId = 'txi4irodvl'
export const apiEndpoint = `https://${apiId}.execute-api.us-east-1.amazonaws.com/prod`
```

Open a terminal.

```ssh
PS C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client> npm install
...
Run `npm audit` for details.
npm notice
npm notice New major version of npm available! 9.8.1 -> 10.8.3
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.8.3
npm notice Run npm install -g npm@10.8.3 to update!
npm notice
```

Now run

```ssh
npm run build
```

Now we have a `build` directory with our compiled code in it:

```sh
PS C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client> ls
    Directory: C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client
Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----         9/30/2024  12:13 PM                build
d-----         9/30/2024  12:12 PM                node_modules
d-----         9/30/2024  11:46 AM                public
d-----         9/30/2024  11:46 AM                src
-a----         9/30/2024   9:02 AM            261 .editorconfig
-a----         9/30/2024   9:02 AM            107 .prettierrc.json
-a----         9/30/2024  11:57 AM         828178 package-lock.json
-a----         9/30/2024   9:02 AM           1090 package.json
-a----         9/30/2024   9:02 AM            791 README.md
-a----         9/30/2024   9:02 AM            494 tsconfig.json
```

Inside build:

```sh
    Directory: C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client\build
Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----         9/30/2024  12:13 PM                static
-a----         9/30/2024  12:13 PM           1035 asset-manifest.json
-a----         9/30/2024   9:02 AM           3870 favicon.ico
-a----         9/30/2024  12:13 PM           2125 index.html
-a----         9/30/2024   9:02 AM            292 manifest.json
-a----         9/30/2024  12:13 PM           2503 precache-manifest.b82a456ee0bc7dc8666f0e1e42e9822f.js
-a----         9/30/2024  12:13 PM           1039 service-worker.js
```

### Upload files to S3 Bucket

Go to the `frontend-website-00` bucket. Under "Objects" select "Upload". Paste all documents inside the `build` directory and click on "Upload".

Under "Objects", click on `index.html` and copy:

Object URL: `https://frontend-website-00.s3.amazonaws.com/index.html`

### Accessing and Using WebApp

Open a browser and go to `https://frontend-website-00.s3.amazonaws.com/index.html`.

<img src="figs/webapp_index.png"  style="width:85%;"/>

We have a form displayed. If we click on it, we see that no items have been added yet:

<img src="figs/items.png"  style="width:25%;"/>

We can add items:

<img src="figs/adding_item.png"  style="width:25%;"/>

We got an error when trying to add a new item:

``` css
Failed to fetch items: Request failed with status code 500
```

Let's check the "logs" in `CloudWatch`:

<img src="figs/error_logs.png"  style="width:85%;"/>

If we expand the ERROR log, we get:

```bash
2024-09-30T16:05:06.899Z	undefined	ERROR	Uncaught Exception 	{
    "errorType": "Runtime.ImportModuleError",
    "errorMessage": "Error: Cannot find module 'aws-sdk'\nRequire stack:\n- /var/task/index.js\n- /var/runtime/index.mjs",
    "stack": [
        "Runtime.ImportModuleError: Error: Cannot find module 'aws-sdk'",
        "Require stack:",
        "- /var/task/index.js",
        "- /var/runtime/index.mjs",
        "    at _loadUserApp (file:///var/runtime/index.mjs:1087:17)",
        "    at async UserFunction.js.module.exports.load (file:///var/runtime/index.mjs:1119:21)",
        "    at async start (file:///var/runtime/index.mjs:1282:23)",
        "    at async file:///var/runtime/index.mjs:1288:1"
    ]
}
```

The module "aws-sdk" hadn't been installed from the template, so we had to do it manually.

Now we can successfully add items:

<img src="figs/filled_form.png"  style="width:70%;"/>

Now we check on `DynamoDB` if they are actually there. Go to "DynamoDB" > "Tables", click on the `all-items` table, click on "Explore items".

<img src="figs/dynamodb_items.png"  style="width:60%;"/>

Likewise, if we delete items on the frontend, they are deleted from the DynamoDB database.

After finished, remove the created Stack (this will delete the API Gateway, the Lambda function and the DynamoDB table) and then remove the S3 Bucket.